# 1.2 - Transformación

El objetivo aquí será transformar las columnas del dataset `listings.csv`

In [1]:
# librerias

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

import re

In [2]:
listings=pd.read_csv('../data/clean_data/listings.csv')

for c in listings.select_dtypes(include='int'):
    listings[c]=pd.to_numeric(listings[c], downcast='integer')

listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21312 entries, 0 to 21311
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            21312 non-null  int32  
 1   host_id                                       21312 non-null  int32  
 2   host_is_superhost                             21312 non-null  int8   
 3   neighbourhood_cleansed                        21312 non-null  object 
 4   neighbourhood_group_cleansed                  21312 non-null  object 
 5   latitude                                      21312 non-null  float64
 6   longitude                                     21312 non-null  float64
 7   property_type                                 21312 non-null  object 
 8   room_type                                     21312 non-null  object 
 9   accommodates                                  21312 non-null 

In [3]:
for c in listings.select_dtypes(include='float'):
    listings[c]=pd.to_numeric(listings[c], downcast='float')

listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21312 entries, 0 to 21311
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            21312 non-null  int32  
 1   host_id                                       21312 non-null  int32  
 2   host_is_superhost                             21312 non-null  int8   
 3   neighbourhood_cleansed                        21312 non-null  object 
 4   neighbourhood_group_cleansed                  21312 non-null  object 
 5   latitude                                      21312 non-null  float32
 6   longitude                                     21312 non-null  float32
 7   property_type                                 21312 non-null  object 
 8   room_type                                     21312 non-null  object 
 9   accommodates                                  21312 non-null 

In [4]:
listings.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,6369,13660,1,Hispanoamérica,Chamartín,40.456280,-3.67763,Apartment,Private room,2,1,1,0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Elevator,Heat...",70,0,5,2,15,1,365,22,52,82,82,73,14,1,0,1,0
1,21853,83531,0,Cármenes,Latina,40.403412,-3.74084,Apartment,Private room,1,1,1,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",17,0,0,1,8,4,40,0,0,0,162,33,0,2,0,2,0
2,23001,82175,0,Legazpi,Arganzuela,40.386951,-3.69304,Apartment,Entire home/apt,6,2,3,5,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",""Wheelcha...",50,300,30,1,10,15,730,2,2,2,213,0,0,6,6,0,0
3,24805,101471,0,Universidad,Centro,40.422020,-3.70395,Apartment,Entire home/apt,3,1,0,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,E...",80,200,30,2,0,5,730,27,57,87,362,9,7,1,1,0,0
4,24836,101653,1,Justicia,Centro,40.419949,-3.69764,Apartment,Entire home/apt,4,1,2,3,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",115,200,0,4,0,3,10,24,54,77,342,67,15,1,1,0,0


**Columnas Numéricas**

Lo primero que haré será determinar que columnas numéricas voy a transformar y cómo.

In [5]:
listings.select_dtypes(np.number).columns

Index(['id', 'host_id', 'host_is_superhost', 'latitude', 'longitude',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms'],
      dtype='object')

+ id: id del airbnb. En realidad es categórico. No se toca.
+ host_id: id del host. En realidad es categórico. No se toca.
+ host_is_superhost: booleano (aunque sea 0-1). No se toca.
+ latitude: latitud del airbnb. 
+ longitude: longitud del airbnb. Ambas columnas se normalizan siguiendo las ecuaciones x=cos(lat) * cos(lon) ; y=cos(lat) * sin(lon) ; z=sin(lat) 
+ accommodates: nº de plazas. Normalizar.
+ bathrooms: nº de baños. Normalizar.
+ bedrooms: nº de habitaciones. Normalizar.
+ beds: nº de camas. Normalizar.
+ price: precio. Variable objetivo. En principio no lo voy a tocar, aunque más adelante probaré a predecir el log-precio, algo habitual en el mundo financiero.
+ security_deposit: fianza. Normalizar.
+ cleaning_fee: gastos de limpieza. Normalizar.
+ guests_included: nº inquilinos. Normalizar.
+ extra_people: dinero por persona extra. Normalizar.
+ minimum_nights: mínimo de noches. Normalizar.
+ maximum_nights: máximo de noches. Normalizar.
+ availability_30: disponibilidad 30 días. Normalizar.
+ availability_60: disponibilidad 60 días. Normalizar.
+ availability_90: disponibilidad 90 días. Normalizar.
+ availability_365: disponibilidad 365 días. Normalizar.
+ number_of_reviews: nº de reviews total. Normalizar.
+ number_of_reviews_ltm: nº de reviews últimos 12 meses. Normalizar.
+ calculated_host_listings_count: nº alquileres por host. Normalizar.
+ calculated_host_listings_count_entire_homes: nº alquileres enteros por host. Normalizar.
+ calculated_host_listings_count_private_rooms: nº alquileres hab. privada por host. Normalizar.
+ calculated_host_listings_count_shared_rooms: nº alquileres hab. compartida por host. Normalizar.

Probaré a normalizar de dos maneras distintas. Por un lado probaré la normalización a través del teorema central del límite (standardization) y también la transformación Box-Cox con distintas lambdas. El resumen de esta transformación es:

Valor Lambda (λ) | Data Transformada(Y’)
--------|------------
-3	| Y-3 = 1/Y3
-2	| Y-2 = 1/Y2
-1	| Y-1 = 1/Y1
-0.5	| Y-0.5 = 1/(√(Y))
0	| log(Y)**
0.5	| Y0.5 = √(Y)
1	| Y$^{1}$ = Y
2	| Y$^{2}$
3   | Y$^{3}$

In [6]:
# normalizaré al final, para probar distintos métodos

cols_to_norm=['accommodates', 'bathrooms', 'bedrooms', 'beds',
              'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
              'minimum_nights', 'maximum_nights', 'availability_30',
              'availability_60', 'availability_90', 'availability_365',
              'number_of_reviews', 'number_of_reviews_ltm',
              'calculated_host_listings_count',
              'calculated_host_listings_count_entire_homes',
              'calculated_host_listings_count_private_rooms',
              'calculated_host_listings_count_shared_rooms']

In [7]:
# x=cos(lat) * cos(lon) ; y=cos(lat) * sin(lon) ; z=sin(lat)
# de momento transformo lat-lon pero no las borro

listings['x']=np.cos(listings.latitude)*np.cos(listings.longitude)
listings['y']=np.cos(listings.latitude)*np.sin(listings.longitude)
listings['z']=np.sin(listings.latitude)

listings.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z
0,6369,13660,1,Hispanoamérica,Chamartín,40.456280,-3.67763,Apartment,Private room,2,1,1,0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Elevator,Heat...",70,0,5,2,15,1,365,22,52,82,82,73,14,1,0,1,0,0.796991,-0.473457,0.375026
1,21853,83531,0,Cármenes,Latina,40.403412,-3.74084,Apartment,Private room,1,1,1,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",17,0,0,1,8,4,40,0,0,0,162,33,0,2,0,2,0,0.748058,-0.510948,0.423488
2,23001,82175,0,Legazpi,Arganzuela,40.386951,-3.69304,Apartment,Entire home/apt,6,2,3,5,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",""Wheelcha...",50,300,30,1,10,15,730,2,2,2,213,0,0,6,6,0,0,0.765575,-0.470904,0.438342
3,24805,101471,0,Universidad,Centro,40.422020,-3.70395,Apartment,Entire home/apt,3,1,0,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,E...",80,200,30,2,0,5,730,27,57,87,362,9,7,1,1,0,0,0.772927,-0.487128,0.406559
4,24836,101653,1,Justicia,Centro,40.419949,-3.69764,Apartment,Entire home/apt,4,1,2,3,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",115,200,0,4,0,3,10,24,54,77,342,67,15,1,1,0,0,0.775268,-0.481796,0.408451


**Columnas Categóricas**

Veamos que transformaciones se harán para cada columna categórica:
    
+ neighbourhood_cleansed: esta columna se deja como esta.
+ neighbourhood_group_cleansed: esta también. La intención es hacer un join más adelante.
+ property_type: tipo de inmueble. Se hará One-Hot encoding (variables dummies)
+ room_type: tipo de alquiler. Se hará One-Hot encoding (variables dummies)
+ bed_type: tipo de cama. Se hará One-Hot encoding (variables dummies)
+ amenities: esta columna es la que más trabajo lleva. Trataré de hacer un One-Hot encoding con todos los amenities para cada uno de los pisos.

In [8]:
listings.select_dtypes('object').columns

Index(['neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'bed_type', 'amenities'],
      dtype='object')

In [9]:
one_hot_columns=['property_type', 'room_type', 'bed_type']

listings=pd.get_dummies(listings, columns=one_hot_columns, drop_first=True)

In [10]:
listings.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_Apartment,property_type_Barn,property_type_Bed and breakfast,property_type_Boutique hotel,property_type_Bungalow,property_type_Camper/RV,property_type_Casa particular (Cuba),property_type_Cave,property_type_Chalet,property_type_Condominium,property_type_Dome house,property_type_Earth house,property_type_Farm stay,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_Hotel,property_type_House,property_type_Hut,property_type_Igloo,property_type_Loft,property_type_Nature lodge,property_type_Other,property_type_Pension (South Korea),property_type_Serviced apartment,property_type_Tent,property_type_Tiny house,property_type_Townhouse,property_type_Villa,property_type_Yurt,room_type_Hotel room,room_type_Private room,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed
0,6369,13660,1,Hispanoamérica,Chamartín,40.456280,-3.67763,2,1,1,0,"{Wifi,""Air conditioning"",Kitchen,Elevator,Heat...",70,0,5,2,15,1,365,22,52,82,82,73,14,1,0,1,0,0.796991,-0.473457,0.375026,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,21853,83531,0,Cármenes,Latina,40.403412,-3.74084,1,1,1,1,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",17,0,0,1,8,4,40,0,0,0,162,33,0,2,0,2,0,0.748058,-0.510948,0.423488,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,23001,82175,0,Legazpi,Arganzuela,40.386951,-3.69304,6,2,3,5,"{TV,Internet,Wifi,""Air conditioning"",""Wheelcha...",50,300,30,1,10,15,730,2,2,2,213,0,0,6,6,0,0,0.765575,-0.470904,0.438342,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,24805,101471,0,Universidad,Centro,40.422020,-3.70395,3,1,0,1,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,E...",80,200,30,2,0,5,730,27,57,87,362,9,7,1,1,0,0,0.772927,-0.487128,0.406559,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,24836,101653,1,Justicia,Centro,40.419949,-3.69764,4,1,2,3,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",115,200,0,4,0,3,10,24,54,77,342,67,15,1,1,0,0,0.775268,-0.481796,0.408451,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


**amenities**

In [11]:
amenities=listings.amenities.apply(lambda x: x.replace('{', '').replace('}', '').replace('"', ''))

In [12]:
all_amenities=set([e for fila in amenities for e in fila.split(',') if e!='' and 'translation missing' not in e])

len(all_amenities)

176

In [13]:
print(all_amenities)

{'Beachfront', 'Private bathroom', 'Patio or balcony', 'First aid kit', 'Oven', 'No stairs or steps to enter', 'Outdoor seating', 'Breakfast table', 'Smart lock', 'Self check-in', 'Ethernet connection', 'Bed linens', 'Bath towel', 'Stove', 'Fixed grab bars for shower', 'Wide entryway', 'Dryer', 'Hammock', 'Washer', 'Wide entrance for guests', 'Beach essentials', 'Body soap', 'Dishes and silverware', 'Hot water kettle', 'Espresso machine', 'Stair gates', 'Accessible-height toilet', 'Refrigerator', 'Free street parking', 'Babysitter recommendations', 'Cleaning before checkout', 'Wide clearance to shower', 'Bedroom comforts', 'Table corner guards', 'Well-lit path to entrance', 'Mudroom', 'Mini fridge', 'Children’s books and toys', 'HBO GO', 'Shared pool', 'Washer / Dryer', 'Sun loungers', 'Gas oven', 'Heated floors', 'Pack ’n Play/travel crib', 'Accessible-height bed', 'Pets live on this property', 'Disabled parking spot', 'Internet', 'Coffee maker', 'Lock on bedroom door', 'Pets allowed'

In [14]:
matrix_amenities=[[1 if e in fila else 0 for e in all_amenities] for fila in amenities]

In [15]:
listings.shape

(21312, 69)

In [16]:
len(matrix_amenities), len(matrix_amenities[0])

(21312, 176)

In [17]:
amenities_df=pd.DataFrame(matrix_amenities, columns=list(all_amenities))

In [18]:
amenities[0]

'Wifi,Air conditioning,Kitchen,Elevator,Heating,Family/kid friendly,Washer,Essentials,Shampoo,Hangers,Hair dryer,Iron,Hot water,Bed linens,Extra pillows and blankets,Pocket wifi'

In [19]:
amenities_df.head()

,Beachfront,Private bathroom,Patio or balcony,First aid kit,Oven,No stairs or steps to enter,Outdoor seating,Breakfast table,Smart lock,Self check-in,Ethernet connection,Bed linens,Bath towel,Stove,Fixed grab bars for shower,Wide entryway,Dryer,Hammock,Washer,Wide entrance for guests,Beach essentials,Body soap,Dishes and silverware,Hot water kettle,Espresso machine,Stair gates,Accessible-height toilet,Refrigerator,Free street parking,Babysitter recommendations,Cleaning before checkout,Wide clearance to shower,Bedroom comforts,Table corner guards,Well-lit path to entrance,Mudroom,Mini fridge,Children’s books and toys,HBO GO,Shared pool,Washer / Dryer,Sun loungers,Gas oven,Heated floors,Pack ’n Play/travel crib,Accessible-height bed,Pets live on this property,Disabled parking spot,Internet,Coffee maker,Lock on bedroom door,Pets allowed,BBQ grill,Memory foam mattress,Waterfront,Elevator,Printer,Fire extinguisher,Warming drawer,Dog(s),Convection oven,Trash can,Beach view,Rain shower,Dishwasher,Safety card,Game console,Gym,Baby monitor,Exercise equipment,TV,Carbon monoxide detector,Wheelchair accessible,Wide entrance,Fixed grab bars for toilet,Outlet covers,Wifi,24-hour check-in,Sound system,Kitchenette,Extra pillows and blankets,Extra space around bed,Crib,Cooking basics,Formal dining area,Long term stays allowed,Balcony,Heated towel rack,Bathtub,Ceiling fan,Hot water,Suitable for events,Fireplace guards,Host greets you,Day bed,Iron,Soaking tub,Paid parking on premises,DVD player,Indoor fireplace,En suite bathroom,Essentials,Room-darkening shades,Keypad,Wide hallways,Private entrance,Air conditioning,Baking sheet,Microwave,Double oven,Smoke detector,Firm mattress,Terrace,Lockbox,Other pet(s),toilet,Shower chair,Window guards,Baby bath,Bathroom essentials,High chair,Garden or backyard,Full kitchen,Private living room,Bathtub with bath chair,Shampoo,Pillow-top mattress,Kitchen,Lake access,Luggage dropoff allowed,Paid parking off premises,Hot tub,Jetted tub,Pocket wifi,Touchless faucets,Wine cooler,Pool,Flat path to guest entrance,Projector and screen,Handheld shower head,Ski-in/Ski-out,Hair dryer,Free parking on premises,Laptop friendly workspace,Walk-in shower,Doorman,Standing valet,Bidet,Single level home,Children’s dinnerware,Other,Steam oven,Smart TV,Breakfast,Family/kid friendly,Barbecue utensils,Building staff,Mobile hoist,Hangers,EV charger,High-resolution computer monitor,Cable TV,Central air conditioning,Heating,Step-free shower,Toilet paper,Cat(s),Netflix,Buzzer/wireless intercom,Smoking allowed,Shower gel,Ground floor access,Amazon Echo,Wide doorway to guest bathroom,Bread maker,Changing table
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,

In [20]:
amenities_df.describe()

,Beachfront,Private bathroom,Patio or balcony,First aid kit,Oven,No stairs or steps to enter,Outdoor seating,Breakfast table,Smart lock,Self check-in,Ethernet connection,Bed linens,Bath towel,Stove,Fixed grab bars for shower,Wide entryway,Dryer,Hammock,Washer,Wide entrance for guests,Beach essentials,Body soap,Dishes and silverware,Hot water kettle,Espresso machine,Stair gates,Accessible-height toilet,Refrigerator,Free street parking,Babysitter recommendations,Cleaning before checkout,Wide clearance to shower,Bedroom comforts,Table corner guards,Well-lit path to entrance,Mudroom,Mini fridge,Children’s books and toys,HBO GO,Shared pool,Washer / Dryer,Sun loungers,Gas oven,Heated floors,Pack ’n Play/travel crib,Accessible-height bed,Pets live on this property,Disabled parking spot,Internet,Coffee maker,Lock on bedroom door,Pets allowed,BBQ grill,Memory foam mattress,Waterfront,Elevator,Printer,Fire extinguisher,Warming drawer,Dog(s),Convection oven,Trash can,Beach view,Rain shower,Dishwasher,Safety card,Game console,Gym,Baby monitor,Exercise equipment,TV,Carbon monoxide detector,Wheelchair accessible,Wide entrance,Fixed grab bars for toilet,Outlet covers,Wifi,24-hour check-in,Sound system,Kitchenette,Extra pillows and blankets,Extra space around bed,Crib,Cooking basics,Formal dining area,Long term stays allowed,Balcony,Heated towel rack,Bathtub,Ceiling fan,Hot water,Suitable for events,Fireplace guards,Host greets you,Day bed,Iron,Soaking tub,Paid parking on premises,DVD player,Indoor fireplace,En suite bathroom,Essentials,Room-darkening shades,Keypad,Wide hallways,Private entrance,Air conditioning,Baking sheet,Microwave,Double oven,Smoke detector,Firm mattress,Terrace,Lockbox,Other pet(s),toilet,Shower chair,Window guards,Baby bath,Bathroom essentials,High chair,Garden or backyard,Full kitchen,Private living room,Bathtub with bath chair,Shampoo,Pillow-top mattress,Kitchen,Lake access,Luggage dropoff allowed,Paid parking off premises,Hot tub,Jetted tub,Pocket wifi,Touchless faucets,Wine cooler,Pool,Flat path to guest entrance,Projector and screen,Handheld shower head,Ski-in/Ski-out,Hair dryer,Free parking on premises,Laptop friendly workspace,Walk-in shower,Doorman,Standing valet,Bidet,Single level home,Children’s dinnerware,Other,Steam oven,Smart TV,Breakfast,Family/kid friendly,Barbecue utensils,Building staff,Mobile hoist,Hangers,EV charger,High-resolution computer monitor,Cable TV,Central air conditioning,Heating,Step-free shower,Toilet paper,Cat(s),Netflix,Buzzer/wireless intercom,Smoking allowed,Shower gel,Ground floor access,Amazon Echo,Wide doorway to guest bathroom,Bread maker,Changing table
count,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000

In [21]:
listings.drop('amenities', axis=1, inplace=True)

listings=pd.concat([listings, amenities_df], axis=1)

In [22]:
listings.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_Apartment,property_type_Barn,property_type_Bed and breakfast,property_type_Boutique hotel,property_type_Bungalow,property_type_Camper/RV,property_type_Casa particular (Cuba),property_type_Cave,property_type_Chalet,property_type_Condominium,property_type_Dome house,property_type_Earth house,property_type_Farm stay,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_Hotel,property_type_House,property_type_Hut,property_type_Igloo,property_type_Loft,property_type_Nature lodge,property_type_Other,property_type_Pension (South Korea),property_type_Serviced apartment,property_type_Tent,property_type_Tiny house,property_type_Townhouse,property_type_Villa,property_type_Yurt,room_type_Hotel room,room_type_Private room,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,Beachfront,Private bathroom,Patio or balcony,First aid kit,Oven,No stairs or steps to enter,Outdoor seating,Breakfast table,Smart lock,Self check-in,Ethernet connection,Bed linens,Bath towel,Stove,Fixed grab bars for shower,Wide entryway,Dryer,Hammock,Washer,Wide entrance for guests,Beach essentials,Body soap,Dishes and silverware,Hot water kettle,Espresso machine,Stair gates,Accessible-height toilet,Refrigerator,Free street parking,Babysitter recommendations,Cleaning before checkout,Wide clearance to shower,Bedroom comforts,Table corner guards,Well-lit path to entrance,Mudroom,Mini fridge,Children’s books and toys,HBO GO,Shared pool,Washer / Dryer,Sun loungers,Gas oven,Heated floors,Pack ’n Play/travel crib,Accessible-height bed,Pets live on this property,Disabled parking spot,Internet,Coffee maker,Lock on bedroom door,Pets allowed,BBQ grill,Memory foam mattress,Waterfront,Elevator,Printer,Fire extinguisher,Warming drawer,Dog(s),Convection oven,Trash can,Beach view,Rain shower,Dishwasher,Safety card,Game console,Gym,Baby monitor,Exercise equipment,TV,Carbon monoxide detector,Wheelchair accessible,Wide entrance,Fixed grab bars for toilet,Outlet covers,Wifi,24-hour check-in,Sound system,Kitchenette,Extra pillows and blankets,Extra space around bed,Crib,Cooking basics,Formal dining area,Long term stays allowed,Balcony,Heated towel rack,Bathtub,Ceiling fan,Hot water,Suitable for events,Fireplace guards,Host greets you,Day bed,Iron,Soaking tub,Paid parking on premises,DVD player,Indoor fireplace,En suite bathroom,Essentials,Room-darkening shades,Keypad,Wide hallways,Private entrance,Air conditioning,Baking sheet,Microwave,Double oven,Smoke detector,Firm mattress,Terrace,Lockbox,Other pet(s),toilet,Shower chair,Window guards,Baby bath,Bathroom essentials,High chair,Garden or backyard,Full kitchen,Private living room,Bathtub with bath chair,Shampoo,Pillow-top mattress,Kitchen,Lake access,Luggage dropoff allowed,Paid parking off premises,Hot tub,Jetted tub,Pocket wifi,Touchless faucets,Wine cooler,Pool,Flat path to guest entrance,Projector and screen,Handheld shower head,Ski-in/Ski-out,Hair dryer,Free parking on premises,Laptop friendly workspace,Walk-in shower,Doorman,Standing valet,Bidet,Single level home,Children’s dinnerware,Other,Steam oven,Smart TV,Breakfast,Family/kid friendly,Barbecue utensils,Building staff,Mobile hoist,Hangers,EV charger,High-resolution computer monitor,Cable TV,Central air conditioning,Heating,Step-free shower,Toilet paper,Cat(s),Netflix,Buzzer/wireless intercom,Smoking allowed,Shower gel,Ground floor access,Amazon Echo,Wide doorway to guest bathroom,Bread maker,Chan

In [23]:
listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21312 entries, 0 to 21311
Columns: 244 entries, id to Changing table
dtypes: float32(5), int16(10), int32(3), int64(176), int8(11), object(2), uint8(37)
memory usage: 33.4 MB


In [24]:
listings.columns=[e.strip().replace(' ', '_').replace('/', '_').replace('-', '_').lower() for e in listings.columns]

listings=listings.rename(columns = lambda x: re.sub('[^A-Za-z0-9_]+', '', x))

In [25]:
listings.to_csv('../data/transform_data/listings.csv', index=False)